# Preprocessing Test Set

Transform test set features as [Notebook 02](02_Preprocessing_and_Feature_Engineering.ipynb) did for the train set.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Load the Data

df in this notebook is the test data

In [3]:
df = pd.read_csv('../kaggledata/test.csv')

In [4]:
df_train_processed = pd.read_csv('../datasets/f_train.csv')

In [5]:
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [6]:
df_train_processed.head()

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Total Bsmt SF,1st Flr SF,2nd Flr SF,...,Attached or BuiltIn Garage,Finished Garage,Fully Paved Drive,New Sale,Has Alley Access,Total SF,1.5P Gr Liv Area,1.5P Total SF,P3 Overall Qual,SalePrice
0,0.0,13517,6,8,1976,2005,289.0,725.0,725,754,...,1,1,1,0,0,2204.0,56879.040419,103470.699543,216,130500
1,43.0,11492,7,5,1996,1997,132.0,913.0,913,1209,...,1,1,1,0,0,3035.0,97750.293340,167200.681443,343,220000
2,68.0,7922,5,7,1953,2007,0.0,1057.0,1057,0,...,0,0,1,0,0,2114.0,34364.693990,97198.032614,125,109000
3,73.0,9802,5,5,2006,2007,0.0,384.0,744,700,...,1,1,1,0,0,1828.0,54872.000000,78156.353241,125,174000
4,82.0,14235,6,8,1900,1993,0.0,676.0,831,614,...,0,0,0,0,0,2121.0,54929.009867,97681.203724,216,138500


In [7]:
df.isna().sum()

Id                0
PID               0
MS SubClass       0
MS Zoning         0
Lot Frontage    160
               ... 
Misc Feature    838
Misc Val          0
Mo Sold           0
Yr Sold           0
Sale Type         0
Length: 80, dtype: int64

In [8]:
df_train_processed.isna().sum().sum()

0

In [9]:
raw_df_shape = df.shape
raw_df_shape

(879, 80)

### Organised Variables

In [10]:
# variables dropped at first cut in Notebook 1
# the only difference is that 'Id' must be preserved for the test set
first_dropped_vars = {'Bldg Type',
 'Bsmt Cond',
 'Bsmt Exposure',
 'Bsmt Qual',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'BsmtFin Type 1',
 'BsmtFin Type 2',
 'Condition 1',
 'Condition 2',
 'Electrical',
 'Exter Cond',
 'Exterior 1st',
 'Exterior 2nd',
 'Fence',
 'Functional',
 'Garage Cond',
 'Garage Qual',
 'Garage Yr Blt',
 'Heating',
 'Land Slope',
 'MS SubClass',
 'Misc Feature',
 'Pool QC',
 'Roof Matl',
 'Street',
 'Utilities'}

In [11]:
NH_dict = {'GrnHill': ['GrnHill'],
 'NH1': ['IDOTRR', 'MeadowV'],
 'NH2': ['Blueste',
  'BrDale',
  'BrkSide',
  'ClearCr',
  'CollgCr',
  'Crawfor',
  'Edwards',
  'Gilbert',
  'Landmrk',
  'Mitchel',
  'NAmes',
  'NPkVill',
  'NWAmes',
  'OldTown',
  'SWISU',
  'Sawyer',
  'SawyerW'],
 'NH3': ['Blmngtn', 'Somerst', 'Timber', 'Veenker'],
 'NH4': ['Greens', 'NoRidge', 'NridgHt', 'StoneBr']}

### Utility Functions

In [12]:
def binarise(df, input_col, positive_class, drop_col=True):
    """
    Converts a categorical variable to a binary variable in a new column.
    The positive_class specified will be the positive label and mapped to 1.
    The values otehr than the positive_class will be and mapped to 0.
    If drop_col is True, the input column is dropped.
    """
    output_col = df[input_col].map(lambda x: 1 if x==positive_class else 0)
    if drop_col == True:
        df.drop(columns=input_col, inplace=True)
    return output_col

In [13]:
# create function to easily check last cols of df for correct data transformation
# ensures new column of interest was properly added to the dataframe 

def lnc(df, n=3):
    """
    Returns last n columns and first 2 rows of data frame.
    """
    return df.iloc[:,-n:].head(2)

## Drop Variables from First Cut

In [14]:
df.drop(columns=first_dropped_vars, inplace=True)

In [15]:
df.shape[1]

53

## Impute Missing Values

Impute Approach from [Notebook 1](01_EDA_and_Cleaning.ipynb)

<details>
    <summary>Table of Imputation Approaches</summary>
    
| Variable | missing amount (train data) | plausible reason for missing | how to handle missing|
|:-------------------|:---------------------|:-------------------------------|:---------------|
| Lot Frontage|330| no street connected|0|
|Alley|1911|no alley access|'No Access', later encoded |
|Mas Vnr Type|22| no Mas Vnr | 'No Vnr', later encoded |
|Mas Vnr Area|22| no Mas Vnr so area is 0|0|
|Bsmt Unf SF |1|no basement |0|
|Total Bsmt SF|1|no basement |0|
|Bsmt Full Bath|2|no basement bath |0|
|Bsmt Half Bath|2|no basement bath |0|
|Fireplace Qu |1000|no fireplace |'No Fireplace', later encoded|
|Garage Type|113|no garage |'No Garage', later encoded|
|Garage Finish|114|no garage |'No Garage', later encoded|
|Garage Cars|1|no garage |0|
|Garage Area|1|no garage |0|
    
</details>

In [16]:
# cols to impute missing with 0
impute_0_lst = ['Lot Frontage', 'Mas Vnr Area', 
            'Bsmt Unf SF', 'Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath',
           'Garage Cars', 'Garage Area']

In [17]:
impute_0 = dict(zip(impute_0_lst,np.zeros(len(impute_0_lst))))

In [18]:
df.fillna(value=impute_0, inplace=True)

In [19]:
# cols to impute missing with string
impute_str = {
    'Alley': 'No Access',
    'Mas Vnr Type': 'No Vnr',
    'Fireplace Qu': 'No Fireplace',
    'Garage Type': 'No Garage',
    'Garage Finish': 'No Garage'    
}

In [20]:
df.fillna(value=impute_str, inplace=True)

In [21]:
df.isna().sum().sum()

0

## Feature Encoding and Transformation

For refersher, seedrop down list below and observations while doing EDA in [Notebook 01]('01_EDA_and_Cleaning.ipynb').

<details>
    <summary>List of Features to Handle</summary>
    
    0. PID should be converted to a binary variable that represents the two groups, PID starting with 5 and PID starting with 9.

    1. Consolidate these basement variables:
    - BsmtFinSF1: Type 1 finished square feet
    - BsmtFinType2: Quality of second finished area (if present)
    - BsmtFinSF2: Type 2 finished square feet
    - BsmtUnfSF: Unfinished square feet of basement area

        Use TotalBsmtSF

    2. Consolidate these bathroom variables:
    - 'Bsmt Full Bath'
    - 'Bsmt Half Bath'
    - 'Full Bath' (full baths above ground)
    - 'Half Bath' (half baths above ground)

        Make new feature of total bathrooms.


    3. Garage year seems useless, garage cars and garage area can be consolidated


    4. For MS Zoning, consider converting to binary of FV and the rest.


    5. For Lot Shape, convert to binary of regular vs the rest/irregular.


    6. For Land Contour, convert to binary to HLS (on a hill) and the rest.


    7. For Lot Config, convert to CulDSac and the rest.


    8. Neighborhoods will need to be remapped to more sensible buckets of categories.


    9. Convert House Style to binary between 1 Floor and the rest.


    10. Convert Roof Style to binary between Hip and the rest.


    11. Convert Mas Vnr Type to Stone Vnr, Vnr (including Stone, basically not 'None') and None (just cols for first two).


    12. Convert Exter Qual to scale of 0-2, 0 for Fa/Po, 1 for TA, 2 for Gd/Ex.


    13. Convert Foundation to binary between PConc and the rest.


    14. Convert Central Air Yes/No to 1/0. 


    15. Convert Heating QC to Excellent and the rest.


    16. Convert Kitchen Qual to 0-2, Ex(2), Gd(1), everything else (0).


    17. Convert Fireplace Qual to 0-2, Ex(2), Gd(1), everything else (0).


    18. Convert Garage Type to Built-In/Attached and everything else.


    19. Convert Garage Finish to Finished (Fin & Rfn) and everthing else (Unf & None).


    20. Convert Paved Drive to Paved (1) for Y and 0 for everthing else.


    21. Convert 'Sale Type' to 'New Sale' (1) and everthing else (0).
    
    22. Convert Alley to No access (0) and everthing else (essentiall has alley access).
    
</details>

#### Encoding PID

PID should be converted to a binary variable that represents the two groups, PID starting with 5 and PID starting with 9.

In [22]:
# PID only has values starting with 5 or 9
df['PID 9'] = df['PID'].map(lambda x: 1 if str(x).startswith('9') else 0)

In [23]:
df['PID 9'].value_counts(normalize=True)

0    0.50967
1    0.49033
Name: PID 9, dtype: float64

In [24]:
df.drop(columns='PID', inplace=True)

#### Consolidate Basement Variables

In [25]:
df.drop(columns='Bsmt Unf SF', inplace=True)

#### Consolidate Bathroom Variables
- 'Bsmt Full Bath'
- 'Bsmt Half Bath'
- 'Full Bath' (full baths above ground)
- 'Half Bath' (half baths above ground)

    Make new feature of total bathrooms.

In [26]:
df['Bath Log'] = np.log(df['Full Bath'] 
                        + df['Half Bath'] * 0.5 
                        + df['Bsmt Full Bath'] * 0.5 
                        + df['Bsmt Half Bath'] * 0.25)

In [27]:
df.drop(columns=['Full Bath', 'Half Bath', 'Bsmt Full Bath', 'Bsmt Half Bath'], inplace=True)

#### Consolidate Garage Area Variables

Garage cars and garage area can be consolidated as they represent very similar information.

In [28]:
df.drop(columns='Garage Cars', inplace=True)

Garage Cars and Garage Area seem to be covering the same information. Garage Cars will be dropped, Garage Area remaining as it is more correlated with the target variable, and it makese sense that someone interested in buying a house would think more about the garage space rather than how many cars it can hold. A garage that holds two cars can vary in size, with the larger one being more desirable.

#### Encode MS Zoning
Convert to binary variable of FV and the rest.

In [29]:
df['Floating Village'] = binarise(df, 'MS Zoning', 'FV')

#### Encode Lot Shape
Convert to binary of regular vs the rest/irregular.

In [30]:
df['Regular Lot Shape'] = binarise(df, 'Lot Shape', 'Reg')

#### Encode Land Contour
Convert to binary to HLS (on a hill) and the rest.

In [31]:
df['Hillside'] = binarise(df, 'Land Contour', 'HLS')

#### Encode Lot Config
Convert to CulDSac and the rest. 
Apart from the observed trend of higher prices with this attribute, the reasoning is that a cul de sac may represent a more quiet and peaceful location, which is plausibly more desirable.

In [32]:
df['CulDSac'] = binarise(df, 'Lot Config', 'CulDSac')

#### Encode Neighborhood
Remap to identified classes of neighborhoods based on EDA. See NH_dict for mapping.

In [33]:
NH_map = {}
for k,v in NH_dict.items():
    for nh in v:
        NH_map.update({nh:k})

NH2 will be the residual class if a new neighborhood is detected in the test set.

In [34]:
def transform_NH(df, drop_input_col=True):
    """
    Processes 'Neighborhood' (nh) column in dataframe.
    First it maps the nh value to its category as specified in the NH_map
    """
    # interim column to hold mapping before dummy creation
    df['NH Class'] = df['Neighborhood'].map(NH_map)
    # any neighborhood not mapped goes into NH2
    df['NH Class'].fillna('NH2', inplace=True)
    # create dummies
    df = pd.get_dummies(df, columns=['NH Class'],
                       prefix='', prefix_sep='')
    if drop_input_col == True:
        # drops original neighborhood column by default
        df.drop(columns='Neighborhood', inplace=True)
    return df

Note we do not use drop_first=True for get_dummies because the test set does not have the GrnHill neighbourhood class, causing the NH1 column to be dropped if we set drop_first=True.

In [35]:
df = transform_NH(df)

#### Encode House Style
Binary between 1 Story and the rest.

In [36]:
df['1 Story'] = binarise(df, 'House Style', '1Story')

#### Encode Roof Style
Binary between Hip and the rest.

In [37]:
df['Hip Roof'] = binarise(df, 'Roof Style', 'Hip')

#### Encode Mas Vnr Type
Convert to Stone Vnr, Vnr (including Stone, basically not 'None') and None.
Just have cols for Stone Vnr (Stone Vnr) and Vnr (everything that is not 'None').

In [38]:
df['Stone Vnr'] = binarise(df, 'Mas Vnr Type', 'Stone', drop_col=False)

In [39]:
df['Has Vnr'] = df['Mas Vnr Type'].map(lambda x: 0 if x=='None' else 1)

In [40]:
df.drop(columns='Mas Vnr Type', inplace=True)

#### Encode Exter Qual 
Convert to scale of 0-2, 
Gd or Ex = 2,
TA = 1,
Fa or Po = 0

In [41]:
df['Exter Qual Num'] = df['Exter Qual'].map({
    'Gd':2,
    'Ex':2,
    'TA':1,
    'Fa':0,
    'Po':0
})

In [42]:
df.drop(columns='Exter Qual', inplace=True)

#### Encode Foundation
Binary between PConc and the rest.

In [43]:
df['PConc Foundation'] = binarise(df, 'Foundation', 'PConc')

#### Encode Central Air 
Convert Yes/No to 1/0.

In [44]:
df['Has Central Air'] = binarise(df, 'Central Air', 'Y')

#### Encode Heating QC
Binary between Excellent and the rest.

In [45]:
df['Excellent Heating'] = binarise(df, 'Heating QC', 'Ex')

#### Encode Kitchen Quality
Convert Kitchen Qual to scale of 0-2.
Ex = 2, 
Gd = 1, 
everything else = 0

In [46]:
df['Kitchen Qual Num'] = df['Kitchen Qual'].map({
    'Gd':1,
    'Ex':2,
    'TA':0,
    'Fa':0,
    'Po':0
})

In [47]:
df.drop(columns='Kitchen Qual', inplace=True)

#### Encode Fireplace Quality
Convert Fireplace Qu to scale of 0-2.
Ex = 2, 
Gd = 1, 
everything else = 0

In [48]:
df['Fireplace Qu Num'] = df['Fireplace Qu'].map({
    'Gd':1,
    'Ex':2,
    'TA':0,
    'Fa':0,
    'Po':0,
    'No Fireplace':0
})

In [49]:
df.drop(columns='Fireplace Qu', inplace=True)

#### Encode Garage Type
Binary of Built-In/Attached and everything else.

In [50]:
df['Attached or BuiltIn Garage'] = df['Garage Type'].map(lambda x: 1 if x=='Attchd' or x=='BuiltIn' else 0)

In [51]:
df.drop(columns='Garage Type', inplace=True)

#### Encode Garage Finish
Binary of Finished (Fin & RFn) and everthing else (Unf & No Garage).

In [52]:
df['Finished Garage'] = df['Garage Finish'].map(lambda x: 1 if x in ['Fin', 'RFn'] else 0)

In [53]:
df.drop(columns='Garage Finish', inplace=True)

#### Encode Paved Drive
Binary of 1 for Y (Paved) and 0 for everthing else ('N' and 'P' which stands for partial).

In [54]:
df['Fully Paved Drive'] = binarise(df, 'Paved Drive', 'Y')

#### Encode 'Sale Type' 
Binary of 1 for 'New Sale' 0 for everthing else.

In [55]:
df['New Sale'] = binarise(df, 'Sale Type', 'New')

#### Encode Alley

In [56]:
df['Has Alley Access'] = df['Alley'].map(lambda x: 0 if x=='No Access' else 1)

In [57]:
df.drop(columns='Alley', inplace=True)

## Consider Applying Functions to Transform Numerical Variables

#### Create Total SF Feature
Adding above grade area and basement area.

In [58]:
df['Total SF'] = df['Gr Liv Area'] + df['Total Bsmt SF']

#### Create power feature for 'Gr Liv Area' and for 'Total SF'

In [59]:
df['1.5P Gr Liv Area']  = df['Gr Liv Area'] ** 1.5

In [60]:
df['1.5P Total SF']  = df['Total SF'] ** 1.5

#### Create squared feature for 'Year Built'

Squaring the feature does not seem to have an effect on establishing a better fit to the target.
What can be observed is that about after the year 2000, there is a wider spread of prices. Likely due to there being much more recorded data in later years than earlier years.

In [61]:
df['After 1999'] = df['Year Built'].map(lambda x: 1 if x >= 2000 else 0)

#### Create cube feature for 'Overall Qual'

In [62]:
df['P3 Overall Qual'] = df['Overall Qual'] ** 3

## Check Data

In [63]:
df.shape

(879, 57)

In [64]:
raw_df_shape

(879, 80)

In [65]:
df_train_processed.shape

(2049, 57)

Check for columns that are in the processed test set not in the processed trained set. Should only see 'Id'.

In [66]:
set(df.columns).difference(set(df_train_processed.columns))

{'Id'}

Check for columns that are in the processed train set not in the processed test set.

In [67]:
set(df_train_processed.columns).difference(set(df.columns))

{'SalePrice'}

Correct. Expect to see SalePrice as a difference because test set has no SalePrice.

In [68]:
df.isna().sum().sum()

0

In [69]:
df.dtypes.unique()

array([dtype('int64'), dtype('float64'), dtype('uint8')], dtype=object)

## Save Data

In [70]:
df.head()

,Id,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Total Bsmt SF,1st Flr SF,...,Attached or BuiltIn Garage,Finished Garage,Fully Paved Drive,New Sale,Has Alley Access,Total SF,1.5P Gr Liv Area,1.5P Total SF,After 1999,P3 Overall Qual
0,2658,69.0,9142,6,8,1910,1950,0.0,1020.0,908,...,0,0,1,0,1,2948.0,84656.545831,160063.098158,0,216
1,2718,0.0,9662,5,4,1977,1977,0.0,1967.0,1967,...,1,1,1,0,0,3934.0,87238.168613,246746.802419,0,125
2,2414,58.0,17104,7,5,2006,2006,0.0,654.0,664,...,1,1,1,1,0,2150.0,57862.526181,99691.398827,1,343
3,1989,60.0,8520,5,6,1923,2006,0.0,968.0,968,...,0,0,0,0,0,1936.0,30117.092024,85184.000000,0,125
4,625,0.0,9500,6,5,1963,1963,247.0,1394.0,1394,...,1,1,1,0,0,2788.0,52046.815311,147210.624182,0,216


In [71]:
df.to_csv('../datasets/f_test.csv', index=False)

In [72]:
!ls ../datasets

f_test.csv         sample_sub_reg.csv train.csv
f_train.csv        test.csv           train_df_c1.csv
